In [9]:
from pytorch_lightning.utilities.seed import seed_everything
import torch
import random
#seed = torch.rand(1)*1000
seed = 715
seed_everything(seed)
torch.manual_seed(seed)



class Experiment():
    def __init__(self,seed,lr_scheduling,LoadPretrained,Weight_quant,Act_quant,Weight_Quant_Enhance,Act_Quant_Enhance,isEWGS_Weights,isEWGS_Act, EWGS_Factor_Weights, EWGS_Factor_Act, RPreLU, Extra_Skip, Two_Step):
        self.seed =  seed
        self.lr = 0.1
        self.lr_scheduling = lr_scheduling
        self.LoadPretrained = LoadPretrained
        self.Weight_quant = Weight_quant
        self.Act_quant = Act_quant
        self.Weight_Quant_Enhance = Weight_Quant_Enhance
        self.Act_Quant_Enhance = Act_Quant_Enhance
        self.isEWGS_Weights = isEWGS_Weights
        self.EWGS_Factor_Weights = EWGS_Factor_Weights
        self.isEWGS_Act = isEWGS_Act
        self.EWGS_Factor_Act = EWGS_Factor_Act
        self.RPreLU = RPreLU
        self.Extra_Skip = Extra_Skip
        self.Two_Step =  Two_Step
            
            
        string2 = Weight_quant+"_"+Weight_Quant_Enhance
        if(isEWGS_Weights == True):
            string2 = string2 + "_EWGS_" + str(EWGS_Factor_Weights)
    
        string3 = Act_quant+"_"+Act_Quant_Enhance
        if(isEWGS_Act == True):
            string3 = string3 + "_EWGS_" + str(EWGS_Factor_Act)
            
        if(Extra_Skip == True):
            string3 = string3 + "_Extra_skip"
            
        if(Two_Step == True):
            string3 = string3 + "_Two_step_Trg"
    
        self.name = "Config_"+str(self.seed)+"_" +str(lr_scheduling)+"_" +str(RPreLU)+"_"+ string2 +"_"+ string3 
        
    def print(self):
        file1 = open("./"+self.name+ ".cfg","w")
        file1.write("[Experiment]" + "\n")
        file1.write("name = "+ str(self.name) + "\n")
        file1.write("seed = "+ str(self.seed) + "\n")
        file1.write("lr = "+ str(self.lr) + "\n")
        file1.write("lr_scheduling = "+ str(self.lr_scheduling) + "\n")
        file1.write("LoadPretrained = "+ str(self.LoadPretrained) + "\n")

        
        file1.write("[Weight Quantization]" + "\n")
        file1.write("Weight_quant = "+ str(self.Weight_quant) + "\n")
        
        file1.write("[Activation Quantization]" + "\n")
        file1.write("Act_quant = "+ str(self.Act_quant) + "\n")
        
        
        file1.write("[Quantizer Enhancements]" + "\n")
        file1.write("Weight_Quant_Enhance = "+ str(self.Weight_Quant_Enhance) + "\n")
        file1.write("Act_Quant_Enhance = "+ str(self.Act_Quant_Enhance) + "\n")
        file1.write("isEWGS_Weights = "+ str(self.isEWGS_Weights) + "\n")
        file1.write("EWGS_Factor_Weights = "+ str(self.EWGS_Factor_Weights) + "\n")
        file1.write("isEWGS_Act = "+ str(self.isEWGS_Act) + "\n")
        file1.write("EWGS_Factor_Act = "+ str(self.EWGS_Factor_Act) + "\n")

        
        file1.write("[Architectural Changes]" + "\n")        
        file1.write("RPreLU = "+ str(self.RPreLU) + "\n")
        file1.write("Extra_Skip = "+ str(self.Extra_Skip) + "\n")            
        file1.write("Two_Step = "+ str(self.Two_Step) + "\n") 
        
        file1.write("[Teacher Student apporach]" + "\n")       
       
        
        
        
        file1.close()

Global seed set to 715


In [10]:
# code for generating config files
Experiment_list = []
seed = 1009000
lr_scheduling = "Cosine"
Two_Step = False
LoadPretrained = False
isEWGS_Weights = False
isEWGS_Act = False
EWGS_Factor_Act = 0
Extra_Skip = False



for RPreLU in [False]:#[False, True]:
    for Weight_quant in ["TWN","FGQ","ESA"]:
        for Act_quant in ["FATNN"]:        
            for Weight_Quant_Enhance in ["STE","Gradual", "Triangle"]:#, "Triangle", "Gradual"]:
                for Act_Quant_Enhance in ["STE"]:#, "Triangle", "Gradual"]:   
                    EWGS_Factor_Weights = 0
                    ## Extra skip false
                    for run in range(3):
                        exp = Experiment(seed,lr_scheduling, LoadPretrained, Weight_quant,Act_quant,Weight_Quant_Enhance,Act_Quant_Enhance,isEWGS_Weights,isEWGS_Act, EWGS_Factor_Weights, EWGS_Factor_Act, RPreLU, False, Two_Step)
                        Experiment_list.append(exp)
                        seed = seed +1
                
                    
print(len(Experiment_list))

number_of_loops = len(Experiment_list)//3
remaining = len(Experiment_list)%3

print(number_of_loops)
print(remaining)

27
9
0


In [11]:
runfile = open("./runner.sh","w")
for a in range(number_of_loops):
    Experiment_list[a*3].print()
    Experiment_list[a*3+1].print()
    Experiment_list[a*3+2].print()
    runfile.write("sbatch --partition elec.gpu.q --gres gpu:1 train_"+str(a)+".sh\n")
    overallfile = open("./train_"+str(a)+ ".sh","w")
    overallfile.write("#!/usr/bin/env bash"+"\n")
    overallfile.write("#SBATCH --partition=elec.gpu.q"+"\n")
    overallfile.write("#SBATCH --gres=gpu:1"+"\n")
    overallfile.write("python Basis.py --config_custom="+Experiment_list[a*3].name +".cfg &"+"\n")
    overallfile.write("python Basis.py --config_custom="+Experiment_list[a*3+1].name +".cfg &"+"\n")
    overallfile.write("python Basis.py --config_custom="+Experiment_list[a*3+2].name +".cfg &"+"\n")
    overallfile.write("wait"+"\n") 
    overallfile.close()
    

if remaining:
    overallfile = open("./train_"+str(number_of_loops)+ ".sh","w")
    runfile.write("sbatch --partition elec.gpu.q --gres gpu:1 train_"+str(number_of_loops)+".sh\n")
    overallfile.write("#!/usr/bin/env bash"+"\n")
    overallfile.write("#SBATCH --partition=elec.gpu.q"+"\n")
    overallfile.write("#SBATCH --gres=gpu:1"+"\n")
    for b in range(remaining):
        Experiment_list[number_of_loops*3 + b].print()
        overallfile.write("python Basis.py --config_custom="+Experiment_list[number_of_loops*3 + b].name +".cfg &"+"\n")
    overallfile.write("wait"+"\n") 
overallfile.close()
runfile.close()